In [1]:
import tensorflow as tf
import numpy as n
import os
import matplotlib.pyplot as plt

prologue

In [2]:
SAMPLE_DIR = r"C:\Users\USER\Downloads\archive"
TRAIN_DIR = SAMPLE_DIR + r"\Train"
TEST_DIR = SAMPLE_DIR + r"\Test"

NO_CLASSES = 42

index = 20 #0-29
generateData = False #whether or not to generate dataset

In [3]:
#as structured in the dataset, a map of index values along the image resolution
index_resolution_map = {
    0 : (29, 30, 3),
    1 : (30, 30, 3),
    2 : (30, 30, 3),
    3 : (31, 31, 3),
    4 : (30, 32, 3),
    5 : (31, 31, 3),
    6 : (33, 34, 3),
    7 : (34, 35, 3),
    8 : (33, 34, 3),
    9 : (36, 36, 3),
    10 : (35, 36, 3),
    11 : (37, 38, 3),
    12 : (37, 38, 3),
    13 : (40, 39, 3),
    14 : (41, 42, 3),
    15 : (44, 44, 3),
    16 : (46, 44, 3),
    17 : (47, 48, 3),
    18 : (50, 58, 3),
    19 : (53, 51, 3),
    20 : (57, 54, 3),
    21 : (59, 58, 3),
    22 : (63, 61, 3),
    23 : (70, 70, 3),
    24 : (76, 76, 3),
    25 : (85, 86, 3),
    26 : (97, 97, 3),
    27 : (108, 110, 3),
    28 : (124, 127, 3),
    29 : (114, 148, 3),
}

In [4]:
#preparing the training dataset

if generateData:
    for class_no in range(0, 42):

        os.makedirs(rf"{index}\data\train\{class_no}", exist_ok=True)

        for img in os.listdir(TRAIN_DIR + rf"\{class_no}"):
            if (img.endswith(f"{index}.png")):
                src = TRAIN_DIR + rf"\{class_no}\\" + img
                trgt = rf"{index}\data\train\{class_no}\{img}"
            
                with open(src, "rb") as s:
                    with open(trgt, "wb") as t:
                        t.write(s.read())

In [5]:
#preparing the testing dataset by resizing from source to the same size used for testing
if generateData:
    from PIL import Image
    os.makedirs(rf"test\{index}", exist_ok = True)
    for img_name in os.listdir(TEST_DIR):
        if (img_name.endswith(".png")):
            img = Image.open(TEST_DIR + rf"\{img_name}")
            x = index_resolution_map[index][0]; y = index_resolution_map[index][1]
            img = img.resize((x, y), Image.Resampling.LANCZOS)
            img.save(rf"test\{index}\{img_name}")

main

In [6]:
#transforming the dataset
training = tf.keras.utils.image_dataset_from_directory(rf"{index}\data\train",
                                                      labels = "inferred", 
                                                      validation_split=0.2,
                                                      subset="training",
                                                      seed=123,
                                                      label_mode="int",
                                                      image_size=(index_resolution_map[index][0], index_resolution_map[index][1]),
                                                      batch_size=32)

testing = tf.keras.utils.image_dataset_from_directory(rf"{index}\data\train",
                                                      labels = "inferred", 
                                                      validation_split=0.2,
                                                      subset="validation",
                                                      seed=123,
                                                      label_mode="int",
                                                      image_size=(index_resolution_map[index][0], index_resolution_map[index][1]),
                                                      batch_size=32)

Found 1299 files belonging to 42 classes.
Using 1040 files for training.
Found 1299 files belonging to 42 classes.
Using 259 files for validation.


In [7]:
#1. define a cnn 
def create_cnn(): #first naive model used
      inputs = tf.keras.Input(shape=index_resolution_map[index])
      x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
      x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.Flatten()(x)
      outputs = tf.keras.layers.Dense(NO_CLASSES, activation="softmax")(x) #43 Classes

      return tf.keras.Model(inputs=inputs, outputs=outputs)

def create_thiccer_cnn(): #first naive model used
      inputs = tf.keras.Input(shape=index_resolution_map[index])
      x = tf.keras.layers.Rescaling(1./255)(inputs) 
      x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
      #x = tf.keras.layers.MaxPooling2D(pool_size=2)(x)
      #x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
      x = tf.keras.layers.Flatten()(x)
      outputs = tf.keras.layers.Dense(NO_CLASSES, activation="softmax")(x) #43 Classes
      return tf.keras.Model(inputs=inputs, outputs=outputs)
   

plateus over 80% accuracy at 10 epochs. to confirm if this was indeed the model's maximum, i then tested over 30 epochs.

in this scenario, the model indeed hit 100% accuracy at about

in order to gain a better idea of the model's actual performance, we do not infer a single training loop as the model's performance, but rather the average of runs

in this case, there were 5 runs of 20 epochs each. resulting this was 

avg accuracy:-  0.8830673079192638
avg val_accuracy:-  0.7603088794648647

given the threshold is still beneath the 85% requirment, it is obvious increasing epochs is not the path forward. i turned my focus to the other parameters of the training process.

the fluctuating val_accuracy observed during training may be an indication of a too high learning rate. redu



In [ ]:
NO_RUNS = 1
histories = []
accuracies = []
val_accuracies = []

selected_model = 1
models = [create_cnn, create_thiccer_cnn]

for i in range(0, NO_RUNS):
    model = models[selected_model]()
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0005), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    history =  model.fit(training, validation_data = testing, epochs = 1)
    histories.append(history)

for history in histories:

    accuracies.append(history.history["accuracy"])
    val_accuracies.append(history.history["val_accuracy"])
    print("--> ", history.history["accuracy"])
    plt.plot(history.history["accuracy"])
    plt.plot(history.history["val_accuracy"])

    plt.legend(["train", "val"])
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.show()


print("avg accuracy:- ", n.mean(accuracies))
print("avg val_accuracy:- ", n.mean(val_accuracies))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 57, 54, 3)]       0         
                                                                 
 rescaling_3 (Rescaling)     (None, 57, 54, 3)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 55, 52, 32)        896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 27, 26, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 25, 24, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 12, 12, 64)       0         
 g2D)                                                      

KeyboardInterrupt: 